### Imports

In [ ]:
import os
import math
import numpy as np
import pandas as pd
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from time import sleep


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:5


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive  #This is to make a short form for the mydrive location 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [ ]:
!mkdir YoutubeCommentsDataset
!unzip /mydrive/Code-Mix-Datasets/YoutubeCommentsDataset.zip -d YoutubeCommentsDataset

mkdir: cannot create directory ‘YoutubeCommentsDataset’: File exists
Archive:  /mydrive/Code-Mix-Datasets/YoutubeCommentsDataset.zip
replace YoutubeCommentsDataset/Information Regarding Data.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: YoutubeCommentsDataset/Information Regarding Data.txt  
  inflating: YoutubeCommentsDataset/Youtube_Comments_Data.csv  


### Load embedding

In [ ]:
df_embedding = pd.read_csv('/mydrive/Code-Mix-Datasets/Embeddings/hing_emb', sep=" ", quoting=3, header=None, index_col=0,skiprows=1)

In [ ]:
df_embedding.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
nahi,-0.079912,0.522077,0.009111,-0.103475,-0.042439,0.229031,0.360524,0.047913,0.129397,0.184819,0.037808,-0.516295,0.218961,-0.164668,0.180701,-0.386640,-0.068663,0.016217,0.045931,0.186454,-0.083010,-0.081126,0.172391,0.063896,0.047010,-0.069440,0.074594,-0.237348,0.044247,0.060424,-0.156759,-0.452560,0.343487,-0.115351,0.180884,-0.125824,0.189236,-0.144955,-0.339609,0.361464,...,0.149655,0.218875,0.241330,0.223399,-0.238831,-0.255176,-0.217318,0.172820,-0.086227,-0.201589,0.186472,0.172828,0.274141,-0.322574,0.118850,-0.039497,0.558407,-0.062534,0.144667,-0.311163,-0.056226,-0.450764,-0.219975,-0.270082,0.245673,0.245098,-0.251164,-0.048588,-0.308297,-0.325541,0.049488,-0.291750,0.188353,-0.099099,0.271938,0.311072,-0.446256,-0.093532,0.118786,-0.221102
to,0.268486,0.454443,-0.211620,-0.029896,-0.123828,0.119760,0.146776,0.152356,0.136670,-0.086144,-0.050878,0.013576,0.040101,-0.305341,0.180583,-0.021216,-0.220805,-0.273808,-0.055823,0.103470,-0.067804,-0.220657,0.583009,0.041388,0.046626,-0.076103,-0.099535,0.033646,-0.273642,0.041777,0.049301,0.017838,0.120956,-0.157904,0.218910,-0.127773,-0.149346,-0.056826,-0.404863,0.480747,...,-0.103053,0.046495,0.440512,-0.009003,-0.612618,-0.262637,0.072368,0.173662,0.149637,-0.022141,0.092603,-0.187714,0.202985,-0.423206,0.196088,-0.210534,0.215506,-0.164961,0.310179,0.100638,0.222475,-0.188912,-0.183798,-0.131548,0.143710,0.026334,-0.238585,0.075989,-0.113403,-0.373096,0.256750,0.170485,-0.081619,-0.110159,-0.012980,0.211806,-0.197437,0.094797,-0.070250,-0.186570
ki,-0.342798,0.568509,0.049298,-0.111945,-0.312412,0.490620,0.442846,0.053863,0.177461,0.111002,0.312172,-0.602531,0.264498,-0.201628,0.376125,-0.395881,0.124789,-0.473777,0.126820,0.086264,0.170230,0.236199,0.245333,0.342971,-0.014456,-0.012857,0.217287,-0.167022,-0.175880,-0.169058,-0.369584,-0.350195,0.237084,0.131265,-0.046302,-0.109374,-0.025367,0.038076,-0.132978,0.404412,...,0.315271,0.054077,0.225164,0.142596,-0.407171,-0.082441,-0.147046,0.083077,0.065228,0.372545,0.237747,0.228064,0.267732,-0.206860,0.339351,0.162408,0.475990,-0.131715,0.222665,-0.334802,-0.086397,-0.206919,-0.189970,-0.229659,0.076044,-0.197885,0.092336,0.224985,-0.054457,-0.033281,0.491238,-0.074531,0.165319,-0.015633,0.151389,0.332968,-0.311262,0.057184,-0.281329,-0.346765
ke,-0.299528,0.510413,-0.033908,-0.064933,-0.439092,0.359395,0.342511,-0.102557,0.354761,0.632507,0.167711,-0.358377,0.111304,0.085862,0.180126,-0.066881,0.099755,-0.068138,0.302462,-0.110525,0.085888,-0.144101,0.242838,-0.121287,0.384848,-0.145164,-0.028000,-0.206916,0.156869,-0.467113,0.024789,-0.176063,-0.090807,0.113101,-0.039015,-0.156265,0.282400,-0.011621,-0.006955,0.495004,...,0.084984,0.280584,0.178200,0.169110,-0.342354,-0.104760,-0.074415,0.462324,-0.157399,-0.167785,-0.185156,-0.161399,0.015410,-0.401743,0.125092,0.100341,0.459452,-0.061197,0.009943,-0.189876,0.103493,0.035745,0.167451,-0.301508,-0.042029,0.023666,-0.143649,0.070190,-0.000454,-0.118479,0.202599,-0.206487,0.178269,-0.108783,0.322591,0.347032,-0.558663,-0.018923,0.116121,-0.325165
se,-0.435697,0.357603,-0.040995,0.107330,-0.313671,0.268082,0.246396,0.005616,0.089185,0.249072,0.030711,-0.516266,0.153362,-0.085299,0.171405,-0.030755,-0.162353,-0.314957,0.067558,0.356368,0.173713,-0.225910,0.490172,0.066194,0.041909,-0.225645,0.067686,-0.216134,-0.176741,0.163501,0.008431,-0.266124,0.157955,-0.066839,0.118103,0.194242,0.204604,0.208945,-0.222251,0.397446,...,0.321824,0.252558,0.402438,0.194747,-0.520264,0.130659,-0.343827,0.375568,-0.124099,-0.089241,0.073819,-0.067210,-0.131197,-0.376184,0.359206,0.070664,0.396609,-0.182876,0.060587,-0.114624,-0.045065,-0.147356,-0.000136,-0.341763,-0

### Create vocab dictionary

In [ ]:
m = df_embedding.to_numpy()
m.shape

(40599, 100)

In [ ]:
embedding_matrix = df_embedding.to_numpy()

vocab = []

for word in list(df_embedding.index):
  vocab.append(str(word))

vocab_size , vocab_dim = embedding_matrix.shape
vocab_size, vocab_dim

(40599, 100)

In [ ]:
len(vocab)

40599

In [ ]:
word2idx = {w: idx for (idx, w) in enumerate(vocab)}
idx2word = {idx: w for (idx, w) in enumerate(vocab)}

In [ ]:
len(word2idx) , len(idx2word)

(40598, 40599)

In [ ]:
import itertools
dict(itertools.islice(word2idx.items(), 10))

{'bhi': 9,
 'ka': 7,
 'ke': 3,
 'ki': 2,
 'ko': 5,
 'me': 8,
 'nahi': 0,
 'se': 4,
 'the': 6,
 'to': 1}

### Read data

In [ ]:
def sub_one(x):
	return x - 1

In [ ]:
df = pd.read_csv('/content/YoutubeCommentsDataset/Youtube_Comments_Data.csv')
df= df.sample(frac = 1)

df['Labels'] = df['Labels'].apply(sub_one)

train_df = df[:6800]
val_df = df[6800:8300]
test_df = df[8300:]

train_df.head()


train_data =  train_df['commentText'].values
train_labels = train_df['Labels'].values

val_data =  val_df['commentText'].values
val_labels = val_df['Labels'].values

test_data =  test_df['commentText'].values
test_labels = test_df['Labels'].values

print(len(train_data), len(train_labels))
print(len(val_data), len(val_labels))
print(len(test_data), len(test_labels))



6800 6800
1500 1500
1500 1500


In [ ]:
df.head()

,id,commentText,Labels
8083,Ugh4PSCj66Glk3gCoAEC,It is the tastiest samosa ever,1
3553,Ugz97IPpb6wizkGTkUJ4AaABAg,Nice recipe di,1
6741,UghzVKEiApWLmXgCoAEC,Nice,2
4568,UgjRiq4pfN_yCngCoAEC,its vry easy thank you mam,4
5837,UgxYFbPTEyY9HjqGHgJ4AaABAg,man it's really good but jese 1-2 minutes Me k...,5


In [ ]:

# post = pd.read_csv("positive_add1and2.csv",sep=',')
# neg = pd.read_csv("negative_add_1and2.csv",sep=',')

# data = []
# labels = []

# for index, row in post.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(1)


# for index, row in neg.iterrows():
#   if index >= 30000:
#     break
  
#   comment = str(row['tweet'])
#   data.append(comment)
#   labels.append(0)
  

# len(data), len(labels)

# data, labels  = shuffle(data, labels, random_state = 40)

# data[0], labels[0]

### Tokenizer

In [ ]:
tokenizer = get_tokenizer('basic_english')


def tokenized_tensor(data):

  output_tokenized = []

  for sentence in data:
    output = []
    tokenized = tokenizer(sentence)
    
    for word in tokenized:
      if word in word2idx:
        id = word2idx[word]
        output.append(id)
      else:
        word2idx[word] = len(word2idx)
        id = word2idx[word]
        output.append(id)

    output = torch.tensor(output)


    output_tokenized.append(output)

  return output_tokenized

In [ ]:
# tokenized_sequences = tokenized_tensor(data)

train_tokenized_sequences = tokenized_tensor(train_data)

test_tokenized_seuqences = tokenized_tensor(test_data)

val_tokenized_seuquences = tokenized_tensor(val_data)


In [ ]:
print(train_tokenized_sequences[1])

tensor([1345, 4659,  276])


In [ ]:
word2idx['<PAD>'] = len(word2idx)
word2idx['<PAD>']

44838

In [ ]:
len(word2idx)

44839

In [ ]:
## Create embedding matrix

random_init = torch.nn.Parameter(torch.Tensor( (len(word2idx) - vocab_size), vocab_dim))
torch.nn.init.kaiming_uniform_(random_init, a=math.sqrt(5))


new_matrix = np.zeros( (len(word2idx), vocab_dim) )

new_matrix[:vocab_size, :] = embedding_matrix

embedding_matrix = new_matrix

embedding_matrix[vocab_size:, :] = random_init.detach().numpy()

In [ ]:
embedding_matrix.shape

(44839, 100)

In [ ]:
# padded_sequences = pad_sequence(tokenized_sequences, batch_first= True, padding_value=107512)

train_padded_sequences = pad_sequence(train_tokenized_sequences, batch_first= True, padding_value=word2idx['<PAD>'])

val_padded_sequences = pad_sequence(val_tokenized_seuquences, batch_first= True, padding_value=word2idx['<PAD>'])

test_padded_sequences = pad_sequence(test_tokenized_seuqences, batch_first= True, padding_value=word2idx['<PAD>'])

In [ ]:
(train_padded_sequences[1])

tensor([ 1345,  4659,   276, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
        44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 

### Dataset and Data loader 

In [ ]:
class Dataset(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load the text and their corresponding labels into Pytorch tensors
    """
    def __init__(self, sequences, labels):
        self.labels = labels
        self.sequences = sequences

    def __getitem__(self, idx):
        sample = {}
        sequence = self.sequences[idx]
        label = torch.tensor(self.labels[idx])

        try:
            sample["label"] = label
            sample["token"] = sequence
        except Exception as e:
            print(e)
        
        return sample
    
    def __len__(self):
        return len(self.labels)
        

In [ ]:
train_dataset = Dataset(train_padded_sequences, train_labels)

val_dataset = Dataset(val_padded_sequences, val_labels)

test_dataset = Dataset(test_padded_sequences, test_labels)

### Hyper parameters

In [ ]:
## Hyper parameter

vocab_size = len(word2idx)
embed_dim = vocab_dim
seq_len = 192
hidden_size = 512
num_layer = 3
num_class = 7
batch_size = 256

LEARNING_RATE = 1e-3
EPOCHS = 5
CLIP = 0.3

In [ ]:
# # Create datasets
# dataset = Dataset(padded_sequences, labels)

# split = 0.85
# train_size = int(split*len(dataset))
# val_size = len(dataset) - train_size

# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:

## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )

## For testing
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=2,
    shuffle=True,
    drop_last=True
 )


dataloaders = {'Train': train_loader, 'Val': val_loader}



In [ ]:
len(train_loader), len(val_loader), len(test_loader)

(26, 5, 5)

In [ ]:
max_len = -1
for comment in val_dataset:
  sentence = comment['token']
  max_len = max(max_len,sentence.shape[0])

print(max_len)

109


In [ ]:
train_dataset[0]

{'label': tensor(1),
 'token': tensor([   58,    12,     6, 40598,  9834,   811, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838, 44838,
         44838, 44838,

### Model

In [ ]:
class SelfMatchingLayer(nn.Module):

    def __init__(self,  seq_length, embed_dim, **kwargs):

      super(SelfMatchingLayer, self).__init__()

      self.seq_length = seq_length
      self.embed_dim  = embed_dim

      self.P = torch.nn.Parameter(torch.Tensor(self.embed_dim, self.embed_dim))

      self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.P, a=math.sqrt(5))

      
    def forward(self, x):  
      
      # input shape: [batch, seq_len, embed_dim]


      #---------------------------------------------#
      # calculate weight vector a = {e_i . P.Q . e_j}
      #---------------------------------------------#

      out = torch.matmul(x,  self.P)   #out shape: [batch, seq_len, embed_dim]

      out = torch.matmul(out, torch.transpose(x, 1, 2))   #out shape: [batch, seq_len, seq_len]

      out = F.gelu(out)         # apply non linear activation

      #------------------------------------#
      # take row wise mean and apply softmax
      #------------------------------------#
      out = torch.mean(out, 2)  #out shape: [batch, seq_len, seq_len]

      out = torch.softmax(out, 0)     #out shape: [batch, seq_len, seq_len]

      out = out.unsqueeze(1)          #out shape: [batch, 1, seq_len]

      #-------------------------------------------#
      # calculate weighted embedding of every word
      #-------------------------------------------#
      out = torch.matmul(out, x)

      out = out.squeeze(1)

      return out      #out shape: [batch, seq_len]


In [ ]:
word2idx['<PAD>']

44838

In [ ]:
class SelfNet(nn.Module):

    def __init__(self, vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class):
        super(SelfNet, self).__init__()


        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = word2idx['<PAD>'])
        self.embedding.load_state_dict({'weight': torch.from_numpy(embedding_matrix)})
        self.embedding.weight.requires_grad = True

        self.selfnet_layer = SelfMatchingLayer(seq_len, embed_dim)

        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_size, num_layers = num_layer, dropout = 0.3, bidirectional = True, batch_first = True )


        self.fc1 = nn.Linear(2* hidden_size + embed_dim , hidden_size//4)
        self.fc2 = nn.Linear(hidden_size//4, num_class)

        self.dropout = nn.Dropout(0.3)



    def forward(self, input):

        embedded = self.embedding(input)  #out shape = [batch, seq_len, embed_dim] 

        selfmatch_output = self.selfnet_layer(embedded)  #out shape = [batch, seq_len] 

        lstm_out, _ = self.lstm(embedded)     

        lstm_out = lstm_out[:, -1, :]      #out shape = [batch, 2 * hidden_size]      

        concat = torch.cat( (selfmatch_output, lstm_out), 1)     #out shape = [batch, 2 * hidden_size + seq_len ]      

        linear_out = self.dropout(F.relu(self.fc1(concat)))     #out shape = [batch, hidden_size]      

        final_out = self.fc2(linear_out)     #out shape = [batch, 2]      

        return final_out

In [ ]:

### Test
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)
# model = model.to(device)

# for batch in train_loader:
#   x = batch['token'].to(device)
#   out = model(x)
#   print(out.shape)
#   break



In [ ]:
model

SelfNet(
  (embedding): Embedding(44839, 100, padding_idx=44838)
  (selfnet_layer): SelfMatchingLayer()
  (lstm): LSTM(100, 512, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  (fc1): Linear(in_features=1124, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=7, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

### Optimizer and loss

In [ ]:
#optimizer
optimizer = Adam(model.parameters(), lr = LEARNING_RATE, eps=1e-8)
#Loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
#to calculate accuracy

def get_accuracy(preds, labels):
  total_acc = 0.0
  
  for i in range(len(labels)):
    if labels[i] == preds[i]:
      total_acc+=1.0
  
  return total_acc / len(labels)

### Training

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimizer, 'max', factor=0.2, patience=5, threshold=0.0008, verbose = True)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5, verbose = True)


Adjusting learning rate of group 0 to 1.0000e-03.


In [ ]:
PATH = 'models/best_selfnet_mean+gelu_2.pt'

In [ ]:
best_valid_f1 = 0.0000

for epoch in range(0, EPOCHS):
  

    print('-'*50)
    print('Epoch {}/{}'.format(epoch+1, EPOCHS))

    for phase in ['Train', 'Val']:

        loss = 0.0   #epoch loss
        accuracy = 0.0   #epoch accuracy

        y_true = []
        y_pred = []

        if phase == 'Train':
            model.train()
        else:
            model.eval()
        
        with tqdm(dataloaders[phase], unit="batch") as tepoch:

          for batch in tepoch:
            labels = batch["label"]
            text = batch["token"]

            output = model(text)

            loss = criterion(output, labels)

            if phase == 'Train':

                #zero gradients
                optimizer.zero_grad() 

                # Backward pass  (calculates the gradients)
                loss.backward()   

                # gradient clipping
                nn.utils.clip_grad_norm_(model.parameters(), CLIP)    

                optimizer.step()             # Updates the weights    

            sleep(0.1)
            _, preds = output.data.max(1)
            y_pred.extend(preds.tolist())
            y_true.extend(labels.tolist())
            
            batch_acc = get_accuracy(preds.tolist(), labels.tolist())
            
            
            loss += loss.item()
            accuracy+= batch_acc

              
          epoch_loss = loss / (len(dataloaders[phase]))
          epoch_acc = accuracy / (len(dataloaders[phase]))

          print(phase + ":")
          
          
          #print(confusion_matrix(y_true, y_pred))
          pre = precision_score(y_true, y_pred, average='weighted')
          recall = recall_score(y_true, y_pred, average='weighted')
          f1 = f1_score(y_true, y_pred, average='weighted')
          

          print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))
          # save best model
          print()
          
            
          if phase == 'Val':
                
                if f1 > best_valid_f1:
                    best_valid_f1 = f1
                    
                    torch.save(model.state_dict(), PATH)
                    print('Model Saved!')
                    
                scheduler.step(f1)
                    


--------------------------------------------------
Epoch 1/5


  0%|          | 0/26 [00:00<?, ?batch/s]

### Testing

In [ ]:
model = SelfNet( vocab_size, embed_dim, hidden_size, num_layer, seq_len, num_class)

In [ ]:
model.load_state_dict(torch.load(PATH))

model.to(device)


In [ ]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

In [ ]:
model.load_state_dict(torch.load('models/best_selfnet_mean+gelu.pt'))

model.to(device)


In [ ]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))

In [ ]:
loss = 0.0   #epoch loss
accuracy = 0.0   #epoch accuracy

y_true = []
y_pred = []

# set the model to evaluation mode            
model.eval()
        
with tqdm(test_loader, unit="batch") as tepoch:
  for batch in tepoch:
    labels = batch["label"].to(device)
    text = batch["token"].to(device)
    
    with torch.no_grad():
        output = model(text)
    
    
    _, preds = output.data.max(1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())
            
    batch_acc = get_accuracy(preds.tolist(), labels.tolist())

    loss = criterion(output, labels)
            
            
    loss += loss.item()
    accuracy+= batch_acc

    sleep(0.1)

              
epoch_loss = loss / (len(val_loader))
epoch_acc = accuracy / (len(val_loader))
print('')
print("Inference:")
print("")
print(confusion_matrix(y_true, y_pred))
pre = precision_score(y_true, y_pred, average='micro')
recall = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')
print("")

print("F1: {:.4f}, Precision: {:.4f}, Recall : {:.4f}, Accuracy: {:.4f}, Loss: {:.4f}.".format(f1, pre, recall, epoch_acc, epoch_loss))